In [88]:
import warnings; warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import xgboost as xgb
import lightgbm as lgb
import catboost as cb

from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn import metrics

In [97]:
train_df = pd.read_csv('data/flight_delays_train.csv')
test_df = pd.read_csv('data/flight_delays_test.csv')

In [3]:
train_df.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,dep_delayed_15min
0,c-8,c-21,c-7,1934,AA,ATL,DFW,732,N
1,c-4,c-20,c-3,1548,US,PIT,MCO,834,N
2,c-9,c-2,c-5,1422,XE,RDU,CLE,416,N
3,c-11,c-25,c-6,1015,OO,DEN,MEM,872,N
4,c-10,c-7,c-6,1828,WN,MDW,OMA,423,Y


In [98]:
cols = ["Month", "DayofMonth", "DayOfWeek", "UniqueCarrier", "Origin", "Dest"]
for item in cols:
    train_df[item] = train_df[item].astype("category").cat.codes +1
    test_df[item] = test_df[item].astype("category").cat.codes +1

In [99]:
X_train, y_train = train_df.drop(["dep_delayed_15min"], axis=1), train_df['dep_delayed_15min'].map({'Y': 1, 'N': 0}).values
X_test = test_df

X_train_part, X_valid, y_train_part, y_valid = train_test_split(X_train, y_train, test_size=0.3, random_state=17)

In [104]:
cate_features_name  = ["Month", "DayofMonth", "DayOfWeek", "UniqueCarrier", "Origin", "Dest"]

def auc(m, train, test): 
    return (metrics.roc_auc_score(y_train_part, m.predict(train)),
                            metrics.roc_auc_score(y_valid, m.predict(test)))

d_train = lgb.Dataset(X_train_part, label=y_train_part)
lg = lgb.LGBMClassifier(silent=False)
params = {"max_depth": 50, "learning_rate" : 0.03, "num_leaves": 500,  "n_estimators": 300}
model2 = lgb.train(params, d_train, categorical_feature=cate_features_name)
auc(model2, X_train_part, X_valid)

# clf = cb.CatBoostClassifier(eval_metric="AUC", one_hot_max_size=50, depth=10, iterations=1000, l2_leaf_reg=9, learning_rate=0.15)
# clf.fit(X_train_part, y_train_part, cat_features=cat_features_index)
# auc(clf, X_train_part, X_valid)

(0.9997532250632214, 0.7267071407470489)

In [ ]:
# print(train_df_new.columns[9:].values)

train_columns = np.insert(train_df_new.columns[9:].values, 0, ['Distance']).tolist()
test_columns = np.insert(test_df_new.columns[8:].values, 0, ['Distance']).tolist()

same_columns = list(set(train_columns).intersection(test_columns))
for train in train_columns:
    if train not in same_columns:
        train_columns.remove(train)
for test in test_columns:
    if test not in same_columns:
        test_columns.remove(test)

print(len(train_columns))
print(len(test_columns))

In [85]:
# Prepare to Simple LogitReg
# X_train, y_train = train_df_new[train_columns].values, train_df_new['dep_delayed_15min'].map({'Y': 1, 'N': 0}).values
# X_test = test_df_new[test_columns].values

# X_train_part, X_valid, y_train_part, y_valid = train_test_split(X_train, y_train, test_size=0.3, random_state=17)

In [78]:
# logit_pipe = Pipeline([('scaler', StandardScaler()),
#                        ('logit', LogisticRegression(C=1, random_state=17, solver='liblinear'))])

# logit_pipe = Pipeline([('scaler', StandardScaler()),
#                        ('classifier', XGBClassifier(gamma=0.5, learning_rate=0.1, max_depth=7, n_jobs=4, nthread=1))])

# logit_pipe = Pipeline([('scaler', StandardScaler()),
#                        ('classifier', lgb.LGBMClassifier())])

In [86]:
# logit_pipe.fit(X_train_part, y_train_part)
# logit_valid_pred = logit_pipe.predict_proba(X_valid)[:, 1]
# roc_auc_score(y_valid, logit_valid_pred)

# params = {"classifier__learning_rate" : [0.01, 0.05, 0.1]}
# grid_search = GridSearchCV(lgb.LGBMClassifier(), n_jobs=4, param_grid=params, scoring="roc_auc", verbose=5)
# grid_search.fit(X_train_part, y_train_part)

# print(gsearch.best_params_, gsearch.best_score_)
# logit_valid_pred = logit_pipe.predict_proba(X_valid)[:, 1]
# roc_auc_score(y_valid, logit_valid_pred)

Fitting 3 folds for each of 3 candidates, totalling 9 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   4 out of   9 | elapsed:    3.2s remaining:    4.1s
[Parallel(n_jobs=4)]: Done   6 out of   9 | elapsed:    4.8s remaining:    2.4s
[Parallel(n_jobs=4)]: Done   9 out of   9 | elapsed:    5.8s finished


GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        importance_type='split', learning_rate=0.1, max_depth=-1,
        min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
        n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
        random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
        subsample=1.0, subsample_for_bin=200000, subsample_freq=0),
       fit_params=None, iid='warn', n_jobs=4,
       param_grid={'classifier__learning_rate': [0.01, 0.05, 0.1]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=5)

In [87]:
# logit_valid_pred = grid_search.predict_proba(X_valid)[:, 1]
# roc_auc_score(y_valid, logit_valid_pred)

0.7152627735359867

In [16]:
logit_pipe.fit(X_train, y_train)
logit_test_pred = logit_pipe.predict_proba(X_test)[:, 1]

pd.Series(logit_test_pred, name='dep_delayed_15min').to_csv('data/logit_2feat.csv', index_label='id', header=True)